# Cancer Screening

Some skeleton code has been provided below. Please follow the structure outlined. Your code should be added to the **process_image()** function. Please leave the final 4 lines of code as is, the filenames should be hardcoded and both images should be handled by the process_image function. The code inside process_image can be removed and is there for example only.

## The Algorithm

Your comments should go here. Please see the rubric for details on how to effectively maximise your marks for comments. Please be aware that in-line comments are not a replacement for initial comments.

Here's a [handy reference](https://guides.github.com/pdfs/markdown-cheatsheet-online.pdf) on how to use GitHub markdown.

In [15]:
#Cormac Smith C17723185
#This program reads in an image of cells, uses canny and contour to find the edges of the cells, then usings bounding rectangle to make a rectangle around the cell. 
#This makes it easier to check if the click is inside the cell or not. It then displays that to the user and writes it to a file

import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image
from easygui import *

def process_image(I, outfilepath):
    def click(event,xc,yc,flags,param):
        if event == cv2.EVENT_LBUTTONDOWN:
            #Used a counter so it doesnt go through again if the click is in mulitple rectangles
            counter = 0
            #Loops through the 35 biggest Hulls
            for i in range(35):
                #Get the top left x and y coordinate along with the height and width
                x,y,w,h=cv2.boundingRect(hull_list[i])
                #get the bottom left corner x and y coordinates
                x2 = x + w
                y2 = y + h
                #If statements to check if the click was inside the Hull
                if counter == 0:
                    if xc < x2 and xc > x:
                        if yc > y and yc < y2:
                            counter =+ 1
                            cv2.destroyWindow("Image")
                            G = cv2.cvtColor(I,cv2.COLOR_BGR2GRAY)
                            #Find an average for the threshold
                            T = np.mean(I) + np.std(I)
                            #Create a mask using thresholding
                            T, B = cv2.threshold(G, thresh = T-10, maxval = 255,type = cv2.THRESH_BINARY)
                            #Create and ROI using the mask
                            ROI = cv2.bitwise_not(I,I,mask=B)
                            #Crop out the selected part of the image
                            C = ROI[y:y2,x:x2]
                            #Make the image an appropriate size
                            S = cv2.resize(C, dsize=(400,320))
                            cv2.imshow("Cell",S) 
                            key = cv2.waitKey(0)
                            cv2.imwrite(outfilepath, S)
                        
    #Simple GUI explaining what to do
    title = "Assignment 2"
    msg = "When the image appears, please click on a cell to isolate it.\nIf nothing happens it means no cell was detected, try clicking on another part of the cell."
    button = "Let's Go"
    msgbox(msg, title, button ) 

    #Initilizing my Hull List
    hull_list = []

    gray = cv2.bilateralFilter(I, 7, 50, 50)
    #Used Canny to detect the edges of the cells
    E = cv2.Canny(gray, 30, 150)

    #Used my canny to find contours
    contours,_ =cv2.findContours(E, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

    #Added the hulls from the contours to their own list
    for i in range(len(contours)):
        hull = cv2.convexHull(contours[i])
        hull_list.append(hull)

    #Sorted the hull list by area
    hull_list = sorted(hull_list, key=cv2.contourArea, reverse=True)

    cv2.namedWindow("Image")
    cv2.setMouseCallback("Image", click)    
    cv2.imshow("Image",I) 
    key = cv2.waitKey(0)


# Please do not change the following lines of code
Colour = cv2.imread('Cervical Colour.jpg')
Mono = cv2.imread('Cervical Mono.jpg')

process_image(Colour, 'Cervical Colour - Final.jpg')

process_image(Mono, 'Cervical Mono - Final.jpg')